# 載入套件

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime

# 偽裝成瀏覽器瀏覽資訊
- 有些網站會偵測我們是使用什麼瀏覽器來查詢資料，如果是沒有瀏覽器的查詢有相當高的機會是用程式在做網路爬蟲
- 因此我們在爬蟲的時候建議養成加上瀏覽器資訊的好習慣!

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}

# 收集公司清單
- 這邊可以依需求調整要抓幾個分頁的資料，因為我想抓全台的資料，所以直接設定抓200個分頁

In [ ]:
urls = []
for i in range(200):
    page = 'https://www.1111.com.tw/job-bank/job-index.asp?si=2&fs=1&ps=100&page={}'.format(str(i+1))
    resp = requests.get(page, headers=headers)
    nurls = list(set(re.findall('www.1111.com.tw/corp/[0-9]{1,}/', resp.text)))
    
    # 如果加載新分頁沒有抓到新的資料，就停止迴圈
    if len(urls) == len(list(set(urls + nurls))):
        break
    urls = list(set(urls + nurls))
    print(len(urls))

100
200
300
400
477
573
649
716
748
809
854
921
1015
1114
1189
1253
1338
1423
1473
1505
1594
1693
1793
1861
1961
2061
2161
2261
2337
2427
2517
2585
2626
2637
2671
2771
2854
2954
3016
3046
3132
3154
3220
3290
3357
3457
3549
3647
3737
3829
3913
3999
4068
4104
4159
4181
4279
4379
4458
4558
4658
4751
4798
4898
4909
4938
4970
5058
5142
5242
5342
5441
5519
5584
5625
5680
5721
5748
5814
5914
6014
6075
6175
6244
6289
6344
6392
6451
6489
6589
6689
6789
6878
6962
7036
7065
7138
7196
7264
7346
7431
7531
7631
7724
7824
7917
7975
8009
8034
8084
8119
8219
8303
8403
8479
8563
8631
8714
8800
8853
8878
8957
8995
9095
9177
9277
9329
9395
9470
9562
9591
9667
9752
9843
9939
10039
10133
10208
10308
10404
10500
10572
10609
10629
10666
10750
10839
10927
11027
11113


In [ ]:
urls[:10]

['www.1111.com.tw/corp/50550941/',
 'www.1111.com.tw/corp/73132063/',
 'www.1111.com.tw/corp/51860745/',
 'www.1111.com.tw/corp/72465479/',
 'www.1111.com.tw/corp/51818364/',
 'www.1111.com.tw/corp/72472673/',
 'www.1111.com.tw/corp/72515570/',
 'www.1111.com.tw/corp/72156920/',
 'www.1111.com.tw/corp/72470650/',
 'www.1111.com.tw/corp/73163003/']

# 爬取個別公司資料
- 網頁中有許多資訊，而我們抓的資料可以分成以下幾個區塊
- 基本資料
- 公司簡介
- 產品服務
- 福利制度
- 職缺數量

In [ ]:
url = 'www.1111.com.tw/corp/9721259/'
resp = requests.get('https://' + url)
print(resp.url)
soup = BeautifulSoup(resp.text)

https://www.1111.com.tw/corp/71724069/


## 基本資料區

In [ ]:
def BasicInfo(soup):
    adress, industry, industry_exp, capital, established, tax_id, emp_num, comp_pho, comp_fax, website, relate_url = [''] * 11
    for i in soup.select('article > ul > li'):
        if '聯絡地址' in i.text:
            adress = i.find('div',{'class':'listContent'}).text
        if '行業別' in  i.text:
            industry = i.find('div',{'class':'listContent'}).text
        if '行業說明' in  i.text:
            industry_exp = i.find('div',{'class':'listContent'}).text
        if '資本額' in  i.text:
            capital = i.find('div',{'class':'listContent'}).text
        if '成立時間' in i.text:
            established = i.find('div',{'class':'listContent'}).text
        if '統一編號' in i.text:
            tax_id = i.find('div',{'class':'listContent'}).text
        if '員工人數' in  i.text:
            emp_num = i.find('div',{'class':'listContent'}).text
        if '公司電話' in  i.text:
            try:
                comp_pho = 'https://www.1111.com.tw/' + i.find('img')['src']
            except:
                comp_pho = i.find('div',{'class':'listContent'}).text
        if '公司傳真' in  i.text:
            try:
                comp_fax = 'https://www.1111.com.tw/' + i.find('img')['src']
            except:
                comp_fax = i.find('div',{'class':'listContent'}).text
        if '網站位址' in  i.text:
            website = i.find('a')['href']
        if '相關連結' in  i.text:
            try:
                relate_url = i.find('a')['href']
            except:
                pass
    return adress, industry, industry_exp, capital, established, tax_id, emp_num, comp_pho, comp_fax, website, relate_url

## 公司簡介

In [ ]:
# 公司簡介
' '.join(i.text for i in soup.select('div[class="container"]')[0])

'公司簡介 \n成立時間 ：西元2017年\n\t經營理念 ：\n\t八斤日式料理堅持選用新鮮自然的食材，製作口味獨到的特有菜色，並以親切、即時、高效率的服務態度，打造舒適、親切、放鬆的用餐環境，征服客戶的胃與「心」，獲廣大消費者青睞，成功點「食」成金。\n\n\t\xa0\n\n '

## 產品服務

In [ ]:
# 產品服務
' '.join(i.text for i in soup.select('div[class="container"]')[1])#.select_one('li').text

## 員工福利

In [ ]:
def Emp_Welfare(soup):
    legal, welfare = [''] * 2
    for i in soup.select('div[id="c3"] > div[class="wrap"] > div > p'):
        if '法定項目' in i.text:
            legal = i.text
        if '福利制度' in i.text:
            welfare = i.text
    return legal, welfare

In [ ]:
legal, welfare = Emp_Welfare(soup)
print(legal)
print(welfare)

## 職缺數量
- 因為職缺數量是在另外一個網址送 requests 查詢回來的，所以在這邊另外寫一個爬蟲抓職缺數量的資料

In [ ]:
def JobCounts(NO):
    url = 'https://www.1111.com.tw/job-bank/company-description-joblist.asp?nNo=' + NO
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    count = soup.find('div',{'id':'eCountTot'}).text
    return count

In [ ]:
JobCounts('9721259')

'20'

## 組合函數
把以上函數組合起來應用，讓我們可以輸入網址就輕鬆整理成表格

In [ ]:
def url2df(url):
    print('https://' + url)
    NO = url.split('/',-1)[-2]
    resp = requests.get('https://' + url)
    soup = BeautifulSoup(resp.text)
    # 公司名稱
    comp_name = soup.select('article > ul > li')[0].text    
    # 基本資訊
    adress, industry, industry_exp, capital, established, tax_id, emp_num, comp_pho, comp_fax, website, relate_url = BasicInfo(soup)
    # 公司簡介
    comp_intro = ' '.join(i.text for i in soup.select('div[class="container"]')[0])
    # 產品服務
    try:
        comp_ser = ' '.join(i.text for i in soup.select('div[class="container"]')[1])
    except:
        comp_ser = ''
    # 員工福利
    legal, welfare = Emp_Welfare(soup)    
    # 職缺數量
    jobcounts = JobCounts(NO)
    ndf = pd.DataFrame([{'編號':NO,
                         '公司名稱': comp_name,
                         '關鍵詞': soup.find('meta',{'name':'keywords'})['content'],
                         '公司簡介':comp_intro,
                         '產品服務':comp_ser,
                         '法定福利':legal,
                         '公司福利':welfare,
                         '職缺數量':jobcounts,
                         '聯絡地址':adress,
                         '行業別':industry,
                         '行業說明':industry_exp,
                         '資本額':capital,
                         '成立時間':established,
                         '統一編號':tax_id,
                         '員工人數':emp_num,
                         '公司電話':comp_pho,
                         '公司傳真':comp_fax,
                         '網站位址':website,
                         '相關連結':relate_url,
                         '連結網址':resp.url,
                         '更新時間':datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}],
                       columns=['編號','公司名稱','關鍵詞','公司簡介','產品服務','法定福利','公司福利','職缺數量','聯絡地址', '行業別','行業說明',
                                '資本額','成立時間','統一編號', '員工人數','公司電話','公司傳真','網站位址','相關連結','連結網址','更新時間'])
    return ndf

In [ ]:
# url = 'www.1111.com.tw/corp/71724069/'
# url = 'www.1111.com.tw/corp/9721259/'
url = 'www.1111.com.tw/corp/69622670/'
url2df(url)

https://www.1111.com.tw/corp/69622670/
福利制度獎 金 類：禮品、績效獎金保 險 類：意外險居 住 類：員工宿舍交 通 類：上下班交通車設 備 類：有休息寢室、交誼廳娛 樂 類：員工定期聚餐、慶生會、尾牙、春酒補 助 類：員工手機配發或補助其 　他：需穿著員工制服、員工在職教育訓練、良好升遷制度


,編號,公司名稱,關鍵詞,公司簡介,產品服務,法定福利,公司福利,職缺數量,聯絡地址,行業別,行業說明,資本額,成立時間,統一編號,員工人數,公司電話,公司傳真,網站位址,相關連結,連結網址,更新時間
0,69622670,恩閤斯物業有限公司,"恩閤斯物業,恩閤斯物業職缺,恩閤斯物業工作,恩閤斯物業徵才,恩閤斯物業招募,恩閤斯物業徵人,...",公司簡介 【瑪克斯關係企業恩閤斯物業有限公司】擁有知名專業團隊\n \n台灣蜘蛛人與瑪克斯電...,目前公司專業服務項目包含：1.外牆磁磚巡檢修復2.防水工程3.外牆矽力康修補4.外牆冷氣修繕...,法定項目勞保、健保、加班費、勞退提撥金、職災保險,福利制度獎 金 類：禮品、績效獎金保 險 類：意外險居 住 類：員工宿舍交 通 類：上下班交...,104,高雄市仁武區澄新街117之4號地圖,照明設備製造、建築╱工程技術服務、不動產經營／開發,外牆修繕高空繩索工程作業，專業燈具代工安裝、維修。,,,42575169,30 人,https://www.1111.com.tw//graphHandle/txtimg.as...,https://www.1111.com.tw//graphHandle/txtimg.as...,,http://www.marx886.com,https://www.1111.com.tw/corp/69622670/,2019-12-29 08:45


## 執行爬蟲
- 將前面收集到的公司清單逐一透過組合好的函數內爬蟲
- 實測抓 1.1 萬則公司的資料要5個小時左右，如果資料量大的話建議引入多線程的方式來抓資料(加速爬資料的效率)

In [ ]:
df = []
fail = []
for i, url in enumerate(urls):
    if i % 100 == 0:
        print(i)
    try:
        ndf = url2df(url)
        df.append(ndf)
    except:
        fail.append(url)

df = pd.concat(df, ignore_index=True)
df

0
https://www.1111.com.tw/corp/50550941/
https://www.1111.com.tw/corp/73132063/
https://www.1111.com.tw/corp/51860745/
https://www.1111.com.tw/corp/72465479/
https://www.1111.com.tw/corp/51818364/
https://www.1111.com.tw/corp/72472673/
https://www.1111.com.tw/corp/72515570/
https://www.1111.com.tw/corp/72156920/
https://www.1111.com.tw/corp/72470650/
https://www.1111.com.tw/corp/73163003/
https://www.1111.com.tw/corp/2132344/
https://www.1111.com.tw/corp/71055221/
https://www.1111.com.tw/corp/70024662/
https://www.1111.com.tw/corp/50519003/
https://www.1111.com.tw/corp/73145790/
https://www.1111.com.tw/corp/10003460/
https://www.1111.com.tw/corp/1060382/
https://www.1111.com.tw/corp/51665532/
https://www.1111.com.tw/corp/67043542/
https://www.1111.com.tw/corp/72529444/
https://www.1111.com.tw/corp/71173683/
https://www.1111.com.tw/corp/69615012/
https://www.1111.com.tw/corp/50631274/
https://www.1111.com.tw/corp/51685643/
https://www.1111.com.tw/corp/73151959/
https://www.1111.com.tw/c

,編號,公司名稱,關鍵詞,公司簡介,產品服務,法定福利,公司福利,職缺數量,聯絡地址,行業別,行業說明,資本額,成立時間,統一編號,員工人數,公司電話,公司傳真,網站位址,相關連結,連結網址,更新時間
0,50550941,東裕冷凍股份有限公司,"東裕冷凍,東裕冷凍職缺,東裕冷凍工作,東裕冷凍徵才,東裕冷凍招募,東裕冷凍徵人,東裕冷凍股份...",公司簡介 東裕冷凍股份有限公司秉持最優質的服務精神，給客戶最安心的保障~\n冷凍庫、冷藏庫是...,新北市冷凍庫出租、冷凍庫出租、低溫倉庫出租、冷藏庫出租、花卉冷藏庫出租、新北市冷藏庫出租、冷...,法定項目勞保、健保、加班費、男性員工陪產假、育嬰假、特別休假、產假、勞退提撥金,,1,新北市三重區光華路20號地圖,倉儲,出租各種冷凍庫、冷藏庫為主,,,,暫不提供員工人數,https://www.1111.com.tw//graphHandle/txtimg.as...,https://www.1111.com.tw//graphHandle/txtimg.as...,http://www.dungyu.com.tw/,,https://www.1111.com.tw/corp/50550941/,2019-12-29 08:48
1,73132063,泓利科技股份有限公司,"泓利科技,泓利科技職缺,泓利科技工作,泓利科技徵才,泓利科技招募,泓利科技徵人,泓利科技股份...",公司簡介 泓利科技股份有限公司\n泓利科技股份有限公司\n泓利科技股份有限公司\n,,,,2,台北市文山區木新路三段278巷11號4樓地圖,精密儀器相關製造,,,,,暫不提供員工人數,https://www.1111.com.tw//graphHandle/txtimg.as...,暫不提供傳真,,,https://www.1111.com.tw/corp/73132063/,2019-12-29 08:48
2,51860745,伍德中醫診所,"伍德中醫診所,伍德中醫診所職缺,伍德中醫診所工作,伍德中醫診所徵才,伍德中醫診所招募,伍德中...",公司簡介 本診所位於民生社區.交通便利.環境優雅.歡迎有服務熱忱的你(妳)加入我們的團隊.,,法定項目勞保、健保、育嬰假、生理假、特別休假、勞退提撥金,福利制度獎 金 類：全勤獎金、員工生日禮金餐 飲 類：伙食津貼娛 樂 類：國內旅遊、國外旅遊...,6,台北市松山區健康路231號地圖,診所,,,,,暫不提供員工人數,暫不提供電話,暫不提供傳真,,,https://www.1111.com.tw/corp/51860745/,2019-12-29 08:48
3,72465479,"佰驟智能有限公司 BOZO INTELLIGENT CO., LTD.","佰驟智能 BOZO INTELLIGENT CO., LTD.,佰驟智能 BOZO INTE...",公司簡介 F106010 五金批發業 F206010 五金零售業 F113010 機械批發業...,,福利制度其 他：法定項目勞保、健保、週休二日、年假獎 金 類：年終獎金、三節獎金、績效獎金...,福利制度其 他：法定項目勞保、健保、週休二日、年假獎 金 類：年終獎金、三節獎金、績效獎金...,14,桃園市平鎮區平東路一段178巷28號地圖,貿易代理商,,,,,暫不提供員工人數,https://www.1111.com.tw//graphHandle/txtimg.as...,暫不提供傳真,,,https://www.1111.com.tw/corp/72465479/,2019-12-29 08:49
4,51818364,新林營造有限公司,"新林營造,新林營造職缺,新林營造工作,新林營造徵才,新林營造招募,新林營造徵人,新林營造有限公司","公司簡介 自民國86年起至今 屬綜合營造業,包括一般住宅及公共工程皆有承攬 及不動產買賣業等...",,法定項目勞保、健保,福利制度獎 金 類：年終獎金保 險 類：員工團保,1,嘉義縣民雄鄉北斗村光明一街343號地圖,建築工程、其它營造,,,,,暫不提供員工人數,https://www.1111.com.tw//graphHandle/txtimg.as...,https://www.1111.com.tw//graphHandle/txtimg.as...,,,https://www.1111.com.tw/corp/51818364/,2019-12-29 08:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11088,51300754,原野登山體育社(原野運動用品),"原野登山體育社(原野運動用品),原野登山體育社(原野運動用品)職缺,原野登山體育社(原野運動...","公司簡介 原野體育用品成立於1984年,擁有20多年對於多種運動用品性能與功能豐富的經驗,以...","以販售專業運動用品為主,流行運動用品為輔,如:專業運動服飾、專業運動鞋類、流行運動服飾、流行...",法定項目勞保、健保、勞退提撥金,福利制度其 他：1.公司產品優惠價2.彈性輪休3.工作滿兩年出國旅游,1,高雄市左營區後昌路605號地圖,百貨相關,,,,,暫不提供員工人數,https://www.1111.com.tw//graphHandle/txtimg.as...,https://www.1111.com.tw//graphHandle/txtimg.as...,http://www.facebook.com/yesports/?rf=151710198...,,https://www.1111.com.tw/corp/51300754/,2019-12-29 14:59
11089,61361100,楊聰食品商行,"楊聰食品商行,楊聰食品商行職缺,楊聰食品商行工作,楊聰食品商行徵才,楊聰食品商行招募,楊聰食...",公司簡介 阿土伯中日法小點心\n\n歡迎加入我們的行列\n\n亦歡迎小點心批發(日淨賺200...,餅乾/點心/食品/ 夯水果... 等等,,福利制度保 險 類：意外險居 住 類：員工宿舍其 他：員工在職教育訓練有供宿/不定時聚餐/...,8,桃園市龜山區復興二路110巷32之1號(靠近林口)地圖,食品什貨零售,阿土伯中日法小點心,,,,暫不提供員工人數,暫不提供電話,https://www.1111.com.tw//graphHandle/txtimg.as...,http://www.小師父.tw/,,https://www.1111.com.tw/corp/61361100/,2019-12-29 14:59
11090,71633404,偉創科技實業社,"偉創科技實業社,偉創科技實業社職缺,偉創科技實業社工作,偉創科技實業社徵才,偉創科技實業社招...","公司簡介 成立時間： 西元2004年 我們為10年的政府立案安防科技公司,專營監視/門禁/防...",,法定項目勞保、健保、特別休假、勞退提撥金,福利制度獎 金 類：全勤獎金、年節獎金、年終獎金、禮品、激勵獎金、績效獎金娛 樂 類：員工定...,3,新北市三重區龍濱路120之6號1樓地圖,電子通訊／電腦週邊批發,,120萬元,,,5 人,https://www.1111.com.tw//graphHandle/txtimg.as...,https://www.1111.com.tw//graphHandle/txtimg.as...,http://www.king-net.com/,,https://www.1111.com.tw/corp/71633404/,2019-12-29 14:59
11091,69432661,款待實業股份有限公司,"款待實業,款待實業職缺,款待實業工作,款待實業徵才,款待實業招募,款待實業徵人,款待實業股份...",公司簡介 \n◊♦◊♦◊♦ 你！不要懷疑，就是你 ♦◊♦◊♦\n \n ...,款待手路菜、頂級汶萊藍蝦、頂級海鮮食材等。,法定項目勞保、健保、特別休假、勞退提撥金,福利制度獎 金 類：年終獎金、禮品、績效獎金娛 樂 類：慶生會、尾牙、春酒其 他：需穿著員...,7,台北市中山區中山北路一段53巷32號地圖,餐廳／餐館,款待餐廳、進口頂級有機食材專賣,,,,暫不提供員工人數,https://www.1111.com.tw//graphHandle/txtimg.as...,https://www.1111.com.tw//graphHandle/txtimg.as...,,https://www.travelerluxe.com/article/desc/1800...,https://www.1111.com.tw/corp/69432661/,2019-12-29 14:59


In [ ]:
fail

['www.1111.com.tw/corp/72472689/',
 'www.1111.com.tw/corp/73163480/',
 'www.1111.com.tw/corp/72566784/',
 'www.1111.com.tw/corp/72565833/',
 'www.1111.com.tw/corp/72565773/',
 'www.1111.com.tw/corp/69420634/',
 'www.1111.com.tw/corp/73153911/',
 'www.1111.com.tw/corp/73154774/',
 'www.1111.com.tw/corp/73150549/',
 'www.1111.com.tw/corp/73074283/',
 'www.1111.com.tw/corp/72564340/',
 'www.1111.com.tw/corp/72041699/',
 'www.1111.com.tw/corp/73137171/',
 'www.1111.com.tw/corp/72428941/',
 'www.1111.com.tw/corp/73163354/',
 'www.1111.com.tw/corp/73076054/',
 'www.1111.com.tw/corp/69648680/',
 'www.1111.com.tw/corp/72515491/',
 'www.1111.com.tw/corp/72524954/',
 'www.1111.com.tw/corp/70473552/']

# 保存資料
- 先跟 Google Drive 空間串接，再將資料存在 Google Drive 上
- 資料放在 Google Drive 上的 Colab Notebooks 資料夾中

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/'
df.to_excel(path + '1111CompList.xlsx')